# Tutorial 2: Training a spiking neural network on a simple vision dataset

Friedemann Zenke (https://fzenke.net)

> For more details on surrogate gradient learning, please see:
> Neftci, E.O., Mostafa, H., and Zenke, F. (2019). Surrogate Gradient Learning in Spiking Neural Networks.
> https://arxiv.org/abs/1901.09948

In Tutorial 1, we have seen how to train a simple multi-layer spiking neural network on a small synthetic dataset. In this tutorial, we will apply what we have learned so far to a slightly larger dataset.
Concretely, we will use the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

In [1]:
# !pip install tonic

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import torch
import torch.nn as nn
import torchvision

In [3]:
torch.__version__

'2.6.0'

In [4]:
# The coarse network structure is dicated by the Fashion MNIST dataset.
nb_inputs  = 34*34 # https://greg-cohen.com/project/datasets/
nb_hidden  = 100
nb_outputs = 10

time_step = 1e-3
nb_steps  = 100

batch_size = 256

In [5]:
torch.backends.mps.is_available()

True

In [6]:
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [7]:
## Load in the N-MNIST dataset
import tonic

train_dataset = tonic.datasets.NMNIST(save_to='./data', train=True, transform=None, target_transform=None)
test_dataset = tonic.datasets.NMNIST(save_to='./data', train=False, transform=None, target_transform=None)

/opt/anaconda3/envs/snn_hybrid/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  9%|▉         | 89085952/1011893601 [10:04<1:44:23, 147331.24it/s]


KeyboardInterrupt: 

In [ ]:
sample = train_dataset[0]
print(sample)


(array([(10, 30,    937, 1), (33, 20,   1030, 1), (12, 27,   1052, 1), ...,
       ( 7, 15, 302706, 1), (26, 11, 303852, 1), (11, 17, 305341, 1)],
      dtype=[('x', '<i8'), ('y', '<i8'), ('t', '<i8'), ('p', '<i8')]), 0)


In [ ]:
import torch.nn.functional as F

def preprocess_spike_events(spike_events, nb_steps=nb_steps, nb_units=nb_inputs):
    """
    Convert spike events into a binned spike train.

    Args:
        spike_events: Structured NumPy array of spike events (x, y, t, p).
        nb_steps: Number of time steps to bin the spikes into.
        nb_units: Number of neurons (x * y resolution).

    Returns:
        A binned spike train of shape (nb_steps, nb_units).
    """
    spike_train = np.zeros((nb_steps, nb_units), dtype=np.float32)
    for x, y, t, p in spike_events:
        if p == 1:  # Only process the event if p indicates a spike
            time_bin = min(int(t / 1000), nb_steps - 1)  # Bin time into steps
            neuron_id = x + y * int(np.sqrt(nb_inputs))  # Flatten 2D coordinates into 1D neuron ID
            spike_train[time_bin, neuron_id] += 1

    return torch.tensor(spike_train, dtype=torch.float32)

def custom_collate_fn(batch, num_classes=10):
    """
    Custom collate function to handle structured data and one-hot encode the labels.

    Args:
        batch: A list of samples (spike_events, label).
        num_classes: Number of classes for one-hot encoding.

    Returns:
        A tuple (spike_tensors, one_hot_labels).
    """
    spike_tensors = []
    labels = []

    for spike_events, label in batch:
        # Preprocess spike events
        spike_tensor = preprocess_spike_events(spike_events)
        spike_tensors.append(spike_tensor)

        # One-hot encode the label
        one_hot_label = F.one_hot(torch.tensor(label, dtype=torch.int64), num_classes=num_classes)
        labels.append(one_hot_label)

    # Stack spike tensors and labels into batches
    spike_tensors = torch.stack(spike_tensors)
    labels = torch.stack(labels)  # Convert list of one-hot labels into a batch
    return spike_tensors, labels


In [ ]:
# REDO - get the representative split for the subset retrieval


In [ ]:
from torch.utils.data import DataLoader, Subset

train_limit
test_limit

train_subset = torch.utils.data.Subset(train_dataset, range(1000))

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=custom_collate_fn,  # Use the custom collate function
    num_workers=4,
    pin_memory=True
)

# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# nb_steps = train_dataset[0][0].shape[0] # Get the number of time steps
nb_inputs = 34 * 34

In [ ]:
print(train_dataset[0][0].shape[0])

5028


Since we are working with spiking neural networks, we ideally want to use a temporal code to make use of spike timing. To that end, we will use a spike latency code to feed spikes to our network.

### Setup of the spiking network model

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

In [ ]:
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

print("init done")

init done


In [ ]:
def plot_voltage_traces(mem, spk=None, dim=(3,5), spike_height=5):
    gs=GridSpec(*dim)
    if spk is not None:
        dat = 1.0*mem
        dat[spk>0.0] = spike_height
        dat = dat.detach().cpu().numpy()
    else:
        dat = mem.detach().cpu().numpy()
    for i in range(np.prod(dim)):
        if i==0: a0=ax=plt.subplot(gs[i])
        else: ax=plt.subplot(gs[i],sharey=a0)
        ax.plot(dat[i])
        ax.axis("off")

We can now run this code and plot the output layer "membrane potentials" below. As desired, these potentials do not have spikes riding on them.

## Training the network

In [ ]:
class SurrGradSpike(torch.autograd.Function):
    """
    Here we implement our spiking nonlinearity which also implements
    the surrogate gradient. By subclassing torch.autograd.Function,
    we will be able to use all of PyTorch's autograd functionality.
    Here we use the normalized negative part of a fast sigmoid
    as this was done in Zenke & Ganguli (2018).
    """

    scale = 100.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we compute a step function of the input Tensor
        and return it. ctx is a context object that we use to stash information which
        we need to later backpropagate our error signals. To achieve this we use the
        ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor we need to compute the
        surrogate gradient of the loss with respect to the input.
        Here we use the normalized negative part of a fast sigmoid
        as this was done in Zenke & Ganguli (2018).
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
        return grad

# here we overwrite our naive spike function by the "SurrGradSpike" nonlinearity which implements a surrogate gradient
spike_fn  = SurrGradSpike.apply

In [ ]:
def run_snn(inputs):
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = []
    spk_rec = []

    # Compute hidden layer activity
    for t in range(nb_steps):
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = out.detach() # We do not want to backprop through the reset
        # print(f"h1[:, t] shape: {h1[:, t].shape}")
        # print(f"syn shape: {syn.shape}")

        new_syn = alpha*syn +h1[:,t]
        new_mem = (beta*mem +syn)*(1.0-rst)

        mem_rec.append(mem)
        spk_rec.append(out)

        mem = new_mem
        syn = new_syn

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

In [ ]:
from tqdm import tqdm  # Import tqdm

import torch.optim as optim

def train(train_loader, lr=2e-3, nb_epochs=10):
    params = [w1,w2]
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))

    log_softmax_fn = nn.LogSoftmax(dim=1)
    loss_fn = nn.NLLLoss()

    loss_hist = []

    # scaler = torch.amp.GradScaler(device=device) if device.type == 'cuda' else None

    for e in range(nb_epochs):
        local_loss = []
        with tqdm(train_loader, desc=f"Epoch {e+1}/{nb_epochs}", unit="batch") as t:
            for x_local, y_local in t:
                x_local = x_local.to(device)
                y_local = y_local.to(device)

                # print(f"Input dimensions: x_local.shape = {x_local.shape}, y_local.shape = {y_local.shape}")

                # # with torch.amp.autocast(device_type=device.type, enabled=(scaler is not None)):  # Mixed Precision

                #     print(f"Output dimensions: output.shape = {output.shape}")

                output, _ = run_snn(x_local)  # Assuming run_snn() is integrated into model
                log_p_y = log_softmax_fn(output)  # LogSoftmax across all classes
                loss_val = loss_fn(log_p_y, y_local)

                # Backward pass
                # if scaler:  # Mixed precision handling
                #     scaler.scale(loss_val).backward()
                #     scaler.step(optimizer)
                #     scaler.update()
                # else:
                loss_val.backward()
                optimizer.step()

                t.set_postfix(loss=loss_val.item())

                local_loss.append(loss_val.item())
        mean_loss = np.mean(local_loss)
        print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        loss_hist.append(mean_loss)

    return loss_hist


def compute_classification_accuracy(data_loader):
    """ Computes classification accuracy on supplied data in batches. """
    accs = []
    for x_local, y_local in data_loader:
        output,_ = run_snn(x_local)
        m,_= torch.max(output,1) # max over time
        _,am=torch.max(m,1)      # argmax over output units
        tmp = np.mean((y_local==am).detach().cpu().numpy()) # compare to labels
        accs.append(tmp)
    return np.mean(accs)

In [ ]:
loss_hist = train(train_loader, lr=2e-4, nb_epochs=30)

Epoch 1/30:   0%|          | 0/235 [00:00<?, ?batch/s]

Input dimensions: x_local.shape = torch.Size([256, 100, 1156]), y_local.shape = torch.Size([256, 10])


Epoch 1/30:   0%|          | 0/235 [00:52<?, ?batch/s]


UnboundLocalError: cannot access local variable 'output' where it is not associated with a value

In [ ]:
plt.figure(figsize=(3.3,2),dpi=150)
plt.plot(loss_hist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
sns.despine()

In [ ]:
print("Training accuracy: %.3f"%(compute_classification_accuracy(train_loader)))
print("Test accuracy: %.3f"%(compute_classification_accuracy(test_loader)))

In [ ]:
def get_mini_batch(data_loader, shuffle=False):
    for ret in data_loader:
        return ret

In [ ]:
x_batch, y_batch = get_mini_batch(test_loader)
output, other_recordings = run_snn(x_batch)
mem_rec, spk_rec = other_recordings

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(mem_rec, spk_rec)

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(output)

In [ ]:
# Let's plot the hiddden layer spiking activity for some input stimuli

nb_plt = 4
gs = GridSpec(1,nb_plt)
fig= plt.figure(figsize=(7,3),dpi=150)
for i in range(nb_plt):
    plt.subplot(gs[i])
    plt.imshow(spk_rec[i].detach().cpu().numpy().T,cmap=plt.cm.gray_r, origin="lower" )
    if i==0:
        plt.xlabel("Time")
        plt.ylabel("Units")

    sns.despine()

In conclusion, we see that already this simple spiking network solves the classification problem with ~85% accuracy, and there is plenty of room left for tweaking. However, the hidden layer activities do not look very biological. Although the network displays population sparseness in that only a subset of neurons are active at any given time, the individual neurons' firing rates are pathologically high. This pathology is not too surprising since we have not incentivized low activity levels in any way. We will create such an incentive to address this issue by activity regularization in one of the next tutorials.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.